# Your First RAG Application: Personal Wellness Assistant

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Personal Wellness Assistant**.

Imagine having an AI assistant that can answer your health and wellness questions based on a curated knowledge base - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading our Wellness Knowledge Base)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Wellness Assistant)
  - 🚧 Activity #1: Enhance Your Wellness Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using a comprehensive Health & Wellness Guide as our sample data. This content covers exercise, nutrition, sleep, stress management, and healthy habits - perfect for building a personal wellness assistant!

In [131]:
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
documents = text_loader.load_documents()
len(documents)

1

In [132]:
print(documents[0][:100])

The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [133]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

21

Let's take a look at some of the documents we've managed to split.

In [134]:
split_documents[0:1]

['The Personal Wellness Guide\nA Comprehensive Resource for Health and Well-being\n\nPART 1: EXERCISE AND MOVEMENT\n\nChapter 1: Understanding Exercise Basics\n\nExercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.\n\nThe four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.\n\nChapter 2: Exercises for Common Problems\n\nLower Back Pain Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [135]:
import os
import openai

In [136]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. Yes, in more recent versions of text-embedding models there is via a parameter passed to the api
2. Matryoshka learning representation to project fine-grained, high-rank representations onto course-grained or lower rank representations. According to the paper's experiments, there is a tradeoff in dimension and accuracy, but for larger dimensions (100s to 1000s there is diminishing returns in accuracy - increaing the dimensions doesnt improve the accuracy very much). Earlier ways of NLP projecting sparse word representations onto lower, denser representations like Latent Semantic Analysis that uses Singular Value Decomposition would probably also work, but less effectively 

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [ ]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
*Sync will mean making 1 request at a time and waiting for a response from the api before submitting another. Each response tends to take a little while to generate. For our large-ish corpus, the number of requests will be fairly high so to complete this process overall will take a long time.
Async is non-blocking, so as soon as an API request is send, we can make another, and another, without waiting for a response first. Effectively all of the requests are being handled by the server in parallel (up to a point) so the total time to complete all requests will be far less.*

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [ ]:
vector_db.search_by_text("What exercises help with lower back pain?", k=3)

[(' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs an

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
Yes set temparature close to zero should reduce the amount of randomness in the output, given the same input text.

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [64]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [ ]:
print(response)

Hello! The best way to write a loop in Python really depends on what you're trying to achieve, but generally, Python's `for` loop is very clean and Pythonic for iterating over sequences like lists, strings, or ranges.

Here's an example of a simple and effective `for` loop:

```python
for i in range(5):
    print(i)
```

This will print the numbers 0 through 4.

If you need to iterate over elements in a list:

```python
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

This is usually preferred over a `while` loop because it's more concise and less error-prone.

If you have a specific scenario in mind, I'd be happy to help you write a loop tailored for it! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [ ]:
RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [66]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt], temperature=0), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [ ]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are some natural remedies for improving sleep quality?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:

Invoke thinking or reasoning mode. There is a parameter that can be referenced as part of the api to determine how hard it shoukd "think"
Or user chain of thought style prompting like "think through the problem step by step, break the problem down into parts before coming up with a final answer" and so on.  

### 🏗️ Activity #1:

Enhance your Personal Wellness Assistant in some way! 

Suggestions are: 

- **PDF Support**: Allow it to ingest wellness PDFs (meal plans, workout guides, medical information sheets)
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for health queries?
- **Metadata Support**: Add metadata like topic categories (exercise, nutrition, sleep) or difficulty levels to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for health content?
- **Multi-Source Ingestion**: Add the capability to ingest content from YouTube health videos, podcasts, or health blogs

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this wellness assistant most useful for your personal health journey.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [ ]:
### YOUR CODE HERE



#### *Scenario*
I'm not doing wellness- I have a different use-case in mind that will be a small part of my Cert Challenge project 
later in the course.
This is a Financial Services application of a company that offers loans to businesses.
For this exercise the will RAG over rules and procedures  documentation to determine 
if the customer meets basic eligibility requirements for a particular type of loan (~ 25 different types) and
 also what the application process lookw like and, what documents they have to supply to support their application
 (varies a lot depending on the loan and company circumstances).

 One limitation is that I am not using internal documentation for commercial confidentiality reasons but 
 publically available information scraped from the customer-facing website, which is short on detail. 
 
 Each page is almost a 1-1 mapping with a particular product (so some of the semantic chunking has already been done for us
 as apposed to having our system do the working of detect the logical separation of products if they were in
 1 long document continuous document) and each would comfortably fit inside a typically sized chunk, without further chunking.

 Therefore, for the purposes of the excercise I'm artificially making the chunk sizes much smaller than they might be so I can observe what happens 
 when a query needs mutiple chunks to provide a complete answer. 

 #### *INPUTS*
 For the purponses of this exercise I pose some scenarios like : 
 "A customer with a limited liability company established more than 3 years ago
 and needs a loan to cover short term cash flow shortage. What loans are they eligable for?"
 or
 "Customer is a registered company will revenues of > $1m annually, needs a loan of 20k to buy new equipment, repayable over 6 months - what are their options?"
 " What documentation is required if a customer with these attributes {attributes} wants a loan {loan_type}"
 
 #### *TESTS*
* chunk size
* number of chunks retrieved (k)
* text v markdown
* lexical search.
 
 
 #### *OUTPUTS/Vibe check*
Can we observe it hallucinating or providing incorrect or incomplete advice?

# Setup

In [ ]:
!uv add docling bm25s

These are the different kinds of loans being ingested - 1 per webpage.

<img src="images/Screenshot 2026-01-18 at 14.07.17.png" />

In [ ]:
from rich.markdown import Markdown
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker

from docling_core.transforms.chunker.tokenizer.openai import OpenAITokenizer
import tiktoken


converter = DocumentConverter()

KB_URL = "https://www.fundingoptions.com/knowledge/"
# Just a few documents
sources = [
    "asset-finance", "bridging-loans", "invoice-finance", "merchant-cash-advance", "revolving-credit-facility", "property-development-finance",
    "business-loans", "working-capital-finance", "short-term-business-loans", "invoice-discounting", "invoice-factoring-vs-invoice-discounting",
    "loans-for-limited-company", "compare-sole-trader-loans-with-funding-options-by-tide", "finance-leases", "debentures-floating-charges",
    "trade-finance", "how-to-apply-for-the-growth-guarantee-scheme", "invoice-factoring", "debt-financing", "commercial-property-finance",
    "find-a-semi-commercial-mortgage-lender", "business-loans-for-women", "unsecured-business-loans", "secured-business-loans", 
    "business-loans-for-bad-credit", "alternative-finance", "auction-finance", "construction-finance", "equity-finance"
    ]

# Scrape the webpages and create the corpus as both text and markdown
# markdown might provide some extra structure for finding good chunks

corpus_plain_txt = []


corpus_docs = []  # Store the actual DoclingDocument objects
for source in sources:
    doc = converter.convert(KB_URL + source).document
    corpus_docs.append(doc)  # Keep the docling document object, not just text
    corpus_plain_txt.append(doc.export_to_text())


In [67]:
RAG_SYSTEM_TEMPLATE = """You are a bank employee who asesses business loan applications for a UK-based bank.

You may only answer questions about business loans based strictly on the provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my  knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response.
- Only include information that is  directly relevant to the question
- I will tip you £1000 for providing the correct answer """

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "extremely brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

In [39]:
def create_and_index_vector_db(chunk_size=200, chunk_overlap=30):
    text_splitter = CharacterTextSplitter(chunk_size= chunk_size, chunk_overlap = chunk_overlap) # artificially small - see my notes above on why
    split_documents = text_splitter.split_texts(corpus_plain_txt)

    vector_db = VectorDatabase()
    vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

    rag_pipeline = RetrievalAugmentedQAPipeline(
        vector_db_retriever=vector_db,
        llm=chat_openai,
        response_style="detailed",
        include_scores=True
    )
    return rag_pipeline


def run_pipeline(query_text, rag_pipeline, k=3):
    # examples we run through each of the variations and print the results!
    result = rag_pipeline.run_pipeline(
        query_text,
        k=k,
        response_length="comprehensive", 
        include_warnings=True,
        confidence_required=True
    )
    return result



# Experiment 1
### Small context, vary num of chunks retreived (k)

In [68]:
rag_pipeline = create_and_index_vector_db(chunk_size = 200, chunk_overlap = 30)


In [41]:

result = run_pipeline("Customer is a registered company will revenues of > $1m annually, needs a loan of 20k to buy new equipment, repayable over 6 months - what are their options",
rag_pipeline, k=3)


Markdown(result['response'])


Based on the provided context, for a registered company with revenues over $1 million annually seeking a £20,000   
loan to buy new equipment repayable over 6 months, the suitable loan options appear to be:                         

 1 Vehicle or machinery finance: According to Source 3, loans can be taken for purposes including "Vehicle or      
   machinery," which would include new equipment purchases. This aligns with the customer's need to buy new        
   equipment.                                                                                                      
 2 Loan size and term: While Source 2 mentions loans often ranging from £50,000 to £20 million for assets and      
   long-term investments, the customer’s requirement is £20,000 repayable over 6 months (short-term). The context  
   does not explicitly confirm availability of smaller loans under £50,000, but the £20,000 request might still be 
   feasible given the company’s strong revenue.                                                                    
 3 Working capital needs: Source 1 mentions businesses needing flexible working capital support and operating in a 
   B2B environment, though it focuses mainly on invoice finance (with 30-90 day terms), which may not be ideal for 
   equipment purchase repayable over 6 months.                                                                     
 4 Possible product: Therefore, machinery finance or a short-term loan specifically for vehicle or equipment       
   purchase seems most appropriate.                                                                                

Summary: The customer can explore a short-term equipment or machinery finance loan, potentially under the vehicle  
or machinery finance category mentioned in Source 3, suited for borrowing £20,000 repayable over 6 months to buy   
new equipment. The company’s strong revenue supports loan viability. However, the exact availability of loans at   
this smaller amount and short term is not explicitly detailed in the context.

✅ pass

In [ ]:
result = run_pipeline("Customer need a loan to cover short term cash flow shortage. They are a sole trader in business for 5 months. What loans are they eligable for?",
rag_pipeline, k=3)
Markdown(result['response'])


Based on the provided context, a sole trader in business for 5 months needing a loan to cover a short-term cash    
flow shortage would be eligible for an unsecured loan and potentially short-term business loans.                   

Specifically:                                                                                                      

 • Source 1 indicates that unsecured loans can be useful for businesses (including SMEs) without significant assets
   to offer as collateral and for covering payroll, VAT, or similar expenses, which aligns with covering short-term
   cash flow shortages.                                                                                            
 • Source 2 mentions that short-term business loans provide fast funding for cash flow, emergencies, or growth, and
   are typically repaid over 3-24 months. This type of loan matches the need for covering a short-term cash flow   
   shortage.                                                                                                       
 • Source 3 notes that secured loans require assets as collateral and indicates unsecured loans are suitable for   
   sole traders with strong income or turnover. While it highlights a requirement for strong income/turnover for   
   unsecured loans, it does not set a minimum trading time.                                                        

Given that the customer is a sole trader in business for only 5 months, which may imply limited trading history,   
unsecured loans and short-term business loans are the best fit. Secured loans may be less accessible unless assets 
are available.                                                                                                     

Therefore, the customer would be eligible for short-term unsecured loans designed for cash flow needs, assuming    
they meet the lender’s criteria on income or turnover. These loans provide flexible finance quickly and can be     
repaid within 3-24 months.

✅ Pass

In [47]:
# Hallucination check - ask a question on a topic that is not provided in the corpus
result = run_pipeline("The Director of a company whants to buy a house to live in. What are their options?",
rag_pipeline, k=5)
Markdown(result['response'])


Based on the provided context, there is no specific information about loan options for a company director who wants
to buy a house to live in personally. The context covers various types of loans primarily related to business or   
investment properties, such as:                                                                                    

 • Loans for buying mixed-use properties like shops with flats above, pubs with owner's accommodation, office      
   blocks with residential units, or converting mixed-use property (Source 1).                                     
 • Buy-to-let mortgages designed for rental properties, either commercial or residential, focusing on projected    
   rental income for affordability (Sources 2 and 3).                                                              
 • Bridging loans for short-term funding for property purchases or development, including refurbishing properties  
   (Sources 4 and 5).                                                                                              

None of these sources mention personal residential property purchases for directors or individuals to live in.     
Therefore, I don't have information about options for a director wanting to buy a house to live in personally in   
the provided context.

✅ Pass

In [43]:
result = run_pipeline("Limited company needs to borrow £500 for marketing costs. What documents do they require to send?", 
rag_pipeline, k=3)
Markdown(result['response'])


Based on the provided context, a limited company applying for a loan (such as £500 for marketing costs) needs to   
provide the following documents:                                                                                   

 • Director ID and address proof                                                                                   
 • Bank statements                                                                                                 
 • Accounts                                                                                                        

This is indicated in Source 2 under documents required: "Documents: director ID & address, bank statements,        
account" (Source 2).                                                                                               

No other specific documentation is mentioned for loans of this amount, but these are the standard documents lenders
require to assess eligibility for limited company loans.

### Comments
❌ FAIL The final answer is incomplete 

In [ ]:
# try again. increase k=6
result = run_pipeline("Limited company needs to borrow £500 for marketing costs. What documents do they require to send?", 
rag_pipeline, k=6)
Markdown(result['response'])

For a limited company applying to borrow £500 for marketing costs, the documents required to submit include:       

 • Director ID and proof of address                                                                                
 • Recent business bank statements                                                                                 

This is supported by multiple sources in the context. Source 2 specifies "director ID & address, bank statements," 
while Sources 4 and 5 also list "Director ID and proof of address" and "Recent business bank statements" as        
required documents for loan applications.                                                                          

Thus, the company should prepare and provide valid identification and proof of address for the director(s), along  
with up-to-date business bank statements as part of their loan application documentation.

if anything, this is worse as less complete than the previous answer

<img src="images/Screenshot 2026-01-18 at 13.58.25.png" />

In [48]:
# This check is the ability to access information that was stored as a table with pros/cons in separate columns
# does this work for plain text extraction (as opposed to markdown?)

result = run_pipeline("what are the pros and cons of short term business loans?",
rag_pipeline, k=3)
Markdown(result['response'])

Based on the provided context, short-term business loans generally come with shorter repayment terms and higher    
interest rates compared to longer-term business loans (Source 3). However, the detailed pros and cons listed in the
context are partial.                                                                                               

From Source 1 and Source 2, there are pros and cons tables mentioned, but the specific details are not fully       
visible in the provided excerpts. The only clear information is that short-term loans have shorter repayment terms 
and higher rates (Source 3).                                                                                       

Therefore, summarising from what is available:                                                                     

Pros of short-term business loans:                                                                                 

 • They have shorter repayment terms (Source 3).                                                                   

Cons of short-term business loans:                                                                                 

 • They generally come with higher interest rates than longer-term loans (Source 3).                               

Unfortunately, the exact detailed pros and cons from the tables in Source 1 and Source 2 are not fully provided in 
the context.                                                                                                       

If more detailed pros and cons are required, I don't have that information based on the current context.

❌ Fail - missing info - here should be at least 5 pros & cons each 

In [ ]:
# Try again with increased K chunks to retreive

result = run_pipeline("what are the pros and cons of short term business loans?",
rag_pipeline, k=6)
Markdown(result['response'])

Based on the provided context, short-term business loans generally come with shorter repayment terms and higher    
rates than longer-term business loan counterparts (Source 3).                                                      

While the specific pros and cons are not fully detailed in the excerpts, it is indicated that there are pros and   
cons outlined for short-term loans in Source 1 and Source 2. Unfortunately, the exact pros and cons are not        
directly provided in the text above.                                                                               

However, the context implies that one con is the higher interest rates and shorter repayment terms associated with 
short-term loans, which can result in higher periodic payments. The pros are not explicitly stated but may         
typically include faster access to funds and flexibility.                                                          

Since the detailed pros and cons are not fully available in the given snippets, I don't have complete information  
about the pros and cons beyond what is quoted here.

❌ Fail - missing info 

# Experiment 2
### larger chunks (400) and overlaps (50)

In [50]:
rag_pipeline = create_and_index_vector_db(chunk_size = 400, chunk_overlap = 50)


In [51]:
result = run_pipeline("what are the pros and cons of short term business loans?",
rag_pipeline, k=3)
Markdown(result['response'])

The pros and cons of short-term business loans based on the provided context are as follows:                       

Pros:                                                                                                              

 • Quick access to funds, possibly within 24-48 hours (Source 1)                                                   
 • Flexible repayment terms ranging from 3-24 months (Source 1)                                                    
 • Useful for emergencies or opportunities requiring urgent capital (Source 2)                                     
 • May be easier to qualify for than longer-term loans (Source 2)                                                  
 • Potential to improve credit if the loan is repaid on time (Source 2)                                            

Cons:                                                                                                              

 • Higher interest rates compared to long-term loans (Source 1)                                                    
 • Larger repayments required due to the shorter loan term (Source 1, Source 2)                                    
 • Can strain cash flow if not managed properly (Source 2)                                                         
 • Not suitable for large, long-term projects (Source 2)                                                           
 • Fees may apply for early repayment of the loan (Source 2)                                                       

In summary, short-term business loans provide quick working capital with flexible but shorter repayment periods,   
which can be beneficial for urgent needs but come with higher costs and potentially larger repayment amounts       
(Sources 1, 2, and 3).

Ok this is more like it - the table is still spanning multiple chunks, but has managed to piece them together fine. 
Perhaps the increased overlap size helped?

✅ Pass - all pros/cons listed

# Experiment 3  
### Different chunking style 
Rather than fixed character width chunks, going to try to chunk based on html tags like section headers, tables, etc. The chunk sizes will vary in size with the hope we are not chunking up mid paragraph or section or table, to preseve meaning better.

In [52]:
tokenizer = OpenAITokenizer(
    tokenizer=tiktoken.encoding_for_model("gpt-4.1-mini"),
    max_tokens=128 * 1024,  
)

In [53]:

chunker = HybridChunker(
    max_tokens=512,
    merge_small_chunks=True,
    respect_structure=True,  # uses headers, tables, lists, etc.
)


chunk_texts = []
for doc in corpus_docs:
    chunk_iter = chunker.chunk(dl_doc=doc)
    for chunk in chunk_iter:
        chunk_texts.append(chunk.text)  # Extract text from each chunk


Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors


In [54]:
# Rebuuild the vectorstore with hybrid chunker 
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(chunk_texts))

rag_pipeline = RetrievalAugmentedQAPipeline(
        vector_db_retriever=vector_db,
        llm=chat_openai,
        response_style="detailed",
        include_scores=True)

In [55]:
result = run_pipeline("what are the pros and cons of short term business loans?",
rag_pipeline, k=3)
Markdown(result['response'])

Based on the provided information, the pros and cons of short-term business loans are:                             

Pros:                                                                                                              

 • Quick access to funds, often within 24-48 hours, allowing businesses to address urgent needs or opportunities   
   promptly. (Source 1)                                                                                            
 • Flexible repayment terms typically ranging from 3 to 24 months, offering some adaptability in managing          
   repayments. (Source 1)                                                                                          
 • Easier qualification compared to some other loan types, which can facilitate obtaining funds. (Source 1)        
 • Potential to improve your credit score if the loan is repaid on time. (Source 1)                                
 • Provides quick working capital for urgent needs without tying the business into long-term commitments. (Source  
   2)                                                                                                              

Cons:                                                                                                              

 • Higher interest rates compared to long-term loans due to the shorter repayment period. (Source 1 & 2)           
 • Larger repayments are required because the loan must be repaid over a shorter term. (Source 1)                  
 • Can strain cash flow if repayments are not managed carefully. (Source 1)                                        
 • Not suitable for large, long-term projects because of the repayment timeframe and amount. (Source 1)            
 • Early repayment may incur fees. (Source 1)                                                                      
 • Repayments are often more frequent, such as weekly or monthly, which may require closer financial management.   
   (Source 2)                                                                                                      

This comprehensive overview reflects the advantages and drawbacks of short-term business loans based on the        
sourcing provided.

✅ Pass

In [62]:
result = run_pipeline("Limited company needs to borrow £500 for marketing costs. What documents do they require to send?", 
rag_pipeline, k=3)
Markdown(result['response'])

For a limited company borrowing £500 for marketing costs, the documents required to be submitted based on the      
context are:                                                                                                       

 • Director ID and address                                                                                         
 • Recent bank statements                                                                                          
 • Accounts or management information                                                                              
 • VAT returns (if registered)                                                                                     
 • Details of existing borrowing                                                                                   

This is supported by all three sources. Source 1 lists "director ID & address, bank statements, accounts/management
info, VAT returns (if registered), existing borrowing details." Sources 2 and 3 largely confirm the same list.     

There is no indication that security or asset details are required for an unsecured marketing cost loan (Source 3  
specifies security/asset details only for secured loans). Therefore, those are not necessary here.

In [ ]:

chunker = HybridChunker(
    max_tokens=512,
    merge_small_chunks=True,
    respect_structure=True,  # uses headers, tables, lists, etc.
)


chunk_texts = []
for doc in corpus_docs:
    chunk_iter = chunker.chunk(dl_doc=doc)
    for chunk in chunk_iter:
        chunk_texts.append(chunk.text)  # Extract text from each chunk


Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors


✅ Pass

# Summary

Webscraped about 25 pages of types of business loans and the eligibility criteria for most

Chunking can sometimes mess up if the split is in the wrong place - middle of a key peice of information

Arbitrary splits such as fixed width chunking can do this


Bigger chunks or more chunks retreived or bigger overlaps between chunks

But Other Forms of chunking that respect the document structures can work better.

